In [34]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import scipy.io as sio
from scipy.signal import resample
import os

import numpy as np
import random

from multilabel_rbm import RBM as MRBM
from rbm import RBM 
from class_rbm import ClassRBM

from sklearn import preprocessing

In [35]:
experiments = os.listdir('../datasets/sample_experiments/')

In [16]:
experiment_samples = {}

for experiment in experiments:
    data = sio.loadmat('../datasets/sample_experiments/' + experiment)
    data = data['Data_dq'][0][0][1][0][0][0]
    downsample_data = []
    for i in range(0,data.shape[0],200):
        downsample_data.append(data[i])
    downsample_data = np.asarray(downsample_data)
    print(data.shape, downsample_data.shape)
    experiment_samples[experiment[:-4]] = preprocessing.scale(downsample_data, axis=1)

((3492633, 7), (17464, 7))
((1090600, 7), (5453, 7))
((3835275, 7), (19177, 7))
((2817307, 7), (14087, 7))
((1637459, 7), (8188, 7))
((2209425, 7), (11048, 7))
((2488471, 7), (12443, 7))
((3458049, 7), (17291, 7))
((13729174, 7), (68646, 7))
((3867079, 7), (19336, 7))
((1636972, 7), (8185, 7))
((1967146, 7), (9836, 7))
((3410960, 7), (17055, 7))
((3759685, 7), (18799, 7))
((1305561, 7), (6528, 7))
((2338492, 7), (11693, 7))
((3691522, 7), (18458, 7))
((3900352, 7), (19502, 7))


In [17]:
print len(experiment_samples.keys())

18


In [18]:
train = []
test = []

stride = 10
window = 60
for exp in experiment_samples.keys()[:14]:
    experiment_sample = experiment_samples[exp]
    
    samples = []
    for i in range(0, experiment_sample.shape[0], stride):
        if experiment_sample[i:i+window,:].shape[0] == window:
            samples.append(experiment_sample[i:i+window,:])
        
    random.shuffle(samples)
    train += samples
    
for exp in experiment_samples.keys()[14:]:
    experiment_sample = experiment_samples[exp]
    
    samples = []
    for i in range(0, experiment_sample.shape[0], stride):
        if experiment_sample[i:i+window,:].shape[0] == window:
            samples.append(experiment_sample[i:i+window,:])
        
    random.shuffle(samples)
    test += samples
    
train = np.asarray(train)
test = np.asarray(test)

In [19]:
print train.shape, test.shape
print train[:,:,0].max(), train[:,:,1].max(), train[:,:,5].max(), train[:,:,3].max(), train[:,:,4].max(), train[:,:,6].max()
print train[:,:,0].min(), train[:,:,1].min(), train[:,:,5].min(), train[:,:,3].min(), train[:,:,4].min(), train[:,:,6].min()

(25417, 60, 7) (4802, 60, 7)
2.438611428668291 1.5102802766209011 1.4090339014985362 1.053519799774938 0.0 1.8102547386212884
-1.0487468792378505 -0.7994930634492071 -1.933521546821148 -1.3039552374145893 -1.9871619510007181 -2.0135376570303425


In [20]:
class FFNet(nn.Module):
    def __init__(self, input_dim):
        super(FFNet, self).__init__()
        self.linear1 = nn.Linear(input_dim, 256)
        self.linear2 = nn.Linear(256, 512)
        self.linear3 = nn.Linear(512, 256)
        self.linear4 = nn.Linear(256, input_dim)

    def forward(self, seq):
        x = seq.view(seq.size()[0], -1)
        out = self.linear1(x)
        out = self.linear2(out)
        out = self.linear3(out)
        out = self.linear4(out)
        base_out = out.view(seq.size()[0], seq.size()[1], seq.size()[2])
        return base_out

In [27]:
class LSTMNet(nn.Module):

    def __init__(self, hidden_dim, layer1_dim):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(3, hidden_dim, batch_first=True) 
        self.linear1 = nn.Linear(hidden_dim, layer1_dim)
        self.linear2 = nn.Linear(layer1_dim, 3)

    def forward(self, seq):
        lstm_out = self.lstm(seq)[0]
        tmp1 = self.linear1(lstm_out)
        _out = self.linear2(tmp1)
        base_out = _out
        return base_out

In [36]:
#model = LSTMNet(32,16).cuda()
model = FFNet(60*3).cuda()

In [29]:
loss_function = nn.MSELoss()
lr = 0.00001
optimizer = optim.SGD(model.parameters(), lr=lr)

In [39]:
for epoch in range(100):
    train = np.random.permutation(train)
    epoch_loss = 0
    for i in range(0, train.shape[0], 256):
        inp = np.concatenate((train[i:i+256, :, 0:2], train[i:i+256, :, 5:6]), axis=2)
        out_true = np.concatenate((train[i:i+256, :, 3:5], train[i:i+256, :, 6:]), axis=2)
        inp = Variable(torch.from_numpy(inp).type(torch.FloatTensor).cuda())
        out_true = Variable(torch.from_numpy(out_true).type(torch.FloatTensor).cuda())
        model.zero_grad()
        out_pred = model(inp)
        loss = loss_function(out_pred, out_true)
        loss.backward()
        optimizer.step()
        
        #print (loss.data[0])
        epoch_loss += loss.data[0]
    
    test_loss = 0
    for i in range(0, test.shape[0], 256):
        inp = np.concatenate((test[i:i+256, :, 0:2], test[i:i+256, :, 5:6]), axis=2)
        out_true = np.concatenate((test[i:i+256, :, 3:5], test[i:i+256, :, 6:]), axis=2)
        out_true = Variable(torch.from_numpy(out_true).type(torch.FloatTensor).cuda())
        inp = Variable(torch.from_numpy(inp).type(torch.FloatTensor).cuda())
        out_pred = model(inp)
        loss = loss_function(out_pred, out_true)
        
        test_loss += loss.data[0]
        
    print ("Epoch " + str(epoch) +  ": Train loss = " + str(epoch_loss.item()/train.shape[0]) + ", Test loss = " + str(test_loss.item()/test.shape[0]))

/home/sv/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()
/home/sv/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0: Train loss = 0.00242554577225, Test loss = 0.00263700302518
Epoch 1: Train loss = 0.00242522819349, Test loss = 0.00263700302518
Epoch 2: Train loss = 0.00242580241671, Test loss = 0.00263700302518
Epoch 3: Train loss = 0.00242534811099, Test loss = 0.00263700302518
Epoch 4: Train loss = 0.0024250685036, Test loss = 0.00263700302518
Epoch 5: Train loss = 0.00242531719359, Test loss = 0.00263700302518
Epoch 6: Train loss = 0.00242547313136, Test loss = 0.00263700302518
Epoch 7: Train loss = 0.00242573352794, Test loss = 0.00263700302518
Epoch 8: Train loss = 0.0024246478168, Test loss = 0.00263700302518
Epoch 9: Train loss = 0.0024260075822, Test loss = 0.00263700302518
Epoch 10: Train loss = 0.00242532829984, Test loss = 0.00263700302518
Epoch 11: Train loss = 0.00242587430718, Test loss = 0.00263700302518
Epoch 12: Train loss = 0.0024251690602, Test loss = 0.00263700302518
Epoch 13: Train loss = 0.00242477553869, Test loss = 0.00263700302518
Epoch 14: Train loss = 0.002425606

In [24]:
mrbm = ClassRBM(180, 180, 512, 2, 0.0001)
loss_function = nn.MSELoss()

In [25]:
for epoch in range(10):
    train = np.random.permutation(train)
    epoch_loss = 0
    for i in range(0, train.shape[0], 256):
        inp = np.concatenate((train[i:i+256, :, 0:2], train[i:i+256, :, 5:6]), axis=2)
        out = np.concatenate((train[i:i+256, :, 3:5], train[i:i+256, :, 6:]), axis=2)
        inp = Variable(torch.from_numpy(inp).type(torch.FloatTensor).cuda())
        out = Variable(torch.from_numpy(out).type(torch.FloatTensor).cuda())
        
        inp = inp.view(inp.size()[0], inp.size()[1] * inp.size()[2])
        out = out.view(out.size()[0], out.size()[1] * out.size()[2])
        
        inp_loss, out_loss = mrbm.contrastive_divergence(inp, out)
        
        loss = loss_function(out, inp)
        
        #print (loss.data[0])
        epoch_loss += loss.data[0]
    
    test_loss = 0
    for i in range(0, test.shape[0], 256):
        inp = np.concatenate((test[i:i+256, :, 0:2], test[i:i+256, :, 5:6]), axis=2)
        out = np.concatenate((test[i:i+256, :, 3:5], test[i:i+256, :, 6:]), axis=2)
        inp = Variable(torch.from_numpy(inp).type(torch.FloatTensor).cuda())
        
        inp = inp.view(inp.size()[0], inp.size()[1] * inp.size()[2])

        out = mrbm.predict(inp)
        loss = loss_function(out, inp)
        
        test_loss += loss.data[0]
        
    print ("Epoch " + str(epoch) +  ": Train loss = " + str(epoch_loss.item()/train.shape[0]) + ", Test loss = " + str(test_loss.item()/test.shape[0]))

/home/sv/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/sv/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0: Train loss = 0.00712146542161, Test loss = 0.00432044573001
Epoch 1: Train loss = 0.00712249920352, Test loss = 0.00444526267221
Epoch 2: Train loss = 0.0071213639645, Test loss = 0.00449704945162
Epoch 3: Train loss = 0.00712857222198, Test loss = 0.00450827071489
Epoch 4: Train loss = 0.00711900463645, Test loss = 0.00451458776459
Epoch 5: Train loss = 0.0071215506696, Test loss = 0.00451990585931
Epoch 6: Train loss = 0.00711963198959, Test loss = 0.0045235144104
Epoch 7: Train loss = 0.00712493957719, Test loss = 0.00452553575509
Epoch 8: Train loss = 0.00712517851168, Test loss = 0.00452650770055
Epoch 9: Train loss = 0.00712629514023, Test loss = 0.00452692356769
